In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer

## 0. Load & process the features

In [2]:
# Load the extracted features for Sleep-EDF-SC-78
# Note: to extract the features run the notebook SleepEDF-SC +/- 30min.ipynb

df_feats = pd.read_parquet("../../features/sleep-edf__cassette_features_ALL__90s.parquet")
print(df_feats.shape)

(416271, 366)


In [3]:
# Add some delta's & ratios to the features (just as in SleepEDF-SC +/- 30min.ipynb)

sigs = list(set(c.split("__")[0] for c in df_feats.columns))

eeg_signals = [d for d in sigs if "EEG" in d]
bands = ["alpha", "beta", "sdelta", "fdelta", "sigma", "theta"]
for eeg_sig in eeg_signals:
    eeg_bands = [c for c in df_feats.columns if c.startswith(eeg_sig) and c.split("__")[1] in bands]
    windows = sorted(set(b.split("__")[-1] for b in eeg_bands))
    for window in windows:
        # Select the spectral powers
        delta = df_feats["__".join([eeg_sig, "sdelta", window])] + df_feats["__".join([eeg_sig, "fdelta", window])]
        fdelta_theta = df_feats["__".join([eeg_sig, "fdelta", window])] + df_feats["__".join([eeg_sig, "theta", window])]
        alpha = df_feats["__".join([eeg_sig, "alpha", window])]
        beta = df_feats["__".join([eeg_sig, "beta", window])]
        theta = df_feats["__".join([eeg_sig, "theta", window])]
        sigma = df_feats["__".join([eeg_sig, "sigma", window])]
        # Calculate the ratios
        df_feats["__".join([eeg_sig, "fdelta+theta", window])] = fdelta_theta.astype("float32")        

        df_feats["__".join([eeg_sig, "alpha/theta", window])] = (alpha / theta).astype("float32")
        df_feats["__".join([eeg_sig, "delta/beta", window])] = (delta / beta).astype("float32")
        df_feats["__".join([eeg_sig, "delta/sigma", window])] = (delta / sigma).astype("float32")
        df_feats["__".join([eeg_sig, "delta/theta", window])] = (delta / theta).astype("float32")

print(df_feats.shape)

df_feats = df_feats[df_feats.label != "Movement time"]
print(df_feats.shape)
df_feats = df_feats[df_feats.label != "Sleep stage ?"]
print(df_feats.shape)
df_feats["label"] = df_feats.label.apply(lambda x: "Sleep stage 3" if x.endswith("4") else x)
print(df_feats.label.value_counts())

(416271, 396)
(416143, 396)
(414813, 396)
Sleep stage W    285286
Sleep stage 2     69132
Sleep stage R     25835
Sleep stage 1     21521
Sleep stage 3     13039
Name: label, dtype: int64


In [4]:
# Perform the temporal shifts (just as in SleepEDF-SC +/- 30min.ipynb)

feats_30s = [f for f in df_feats.columns if "w=30s" in f]
feats_60s = [f for f in df_feats.columns if "w=1m_" in f]
feats_90s = [f for f in df_feats.columns if "w=1m30s" in f]
print(len(feats_30s), len(feats_60s), len(feats_90s))
dfs = []
for psg_file in df_feats.psg_file.unique():
    sub_df = df_feats[df_feats.psg_file == psg_file]

    sub_df = sub_df.merge(
        sub_df[feats_90s].shift(1).add_suffix("_shift=30s"),
        left_index=True,
        right_index=True,
    )
    sub_df = sub_df.drop(columns=feats_90s)

    sub_df = sub_df.merge(
        sub_df[feats_60s].shift(1).add_suffix("_shift=30s"),
        left_index=True,
        right_index=True,
    )

    sub_df = sub_df.merge(sub_df[feats_30s].shift(2).add_suffix("_shift=1m"), left_index=True, right_index=True)
    sub_df = sub_df.merge(sub_df[feats_30s].shift(1).add_suffix("_shift=30s"), left_index=True, right_index=True)
    sub_df = sub_df.merge(sub_df[feats_30s].shift(-1).add_suffix("_shift=-30s"), left_index=True, right_index=True)
    sub_df = sub_df.merge(sub_df[feats_30s].shift(-2).add_suffix("_shift=-1m"), left_index=True, right_index=True)
    dfs += [sub_df]
df_feats = pd.concat(dfs)
df_feats.shape

131 131 131


(414813, 1051)

In [5]:
# Trim the features to include 30 min of wake time before/after sleep period

def get_repeat_length(val, arr):
    if arr[0] != val:
        return 0
    return np.where(arr != val)[0][0] + 1

dfs = []
for psg_file in df_feats["psg_file"].unique():
    sub_df = df_feats[df_feats.psg_file == psg_file]  # .sort_index()
    labels = sub_df["label"].values
    nb_wake_before_sleep = get_repeat_length("Sleep stage W", labels)
    nb_wake_after_sleep = get_repeat_length("Sleep stage W", labels[::-1])
    start_idx = max(0, nb_wake_before_sleep - 30 * 2)
    end_idx = min(-1, -nb_wake_after_sleep + 30 * 2)
    dfs.append(sub_df[start_idx:end_idx])
df_feats = pd.concat(dfs)

print(df_feats.shape)

(195168, 1051)


In [6]:
feat_cols = [c for c in df_feats.columns if c not in ["label", "psg_file", "patient_id"]]
print(len(feat_cols))

1048


In [7]:
## The linear model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import StratifiedGroupKFold, cross_validate

pipe = Pipeline(
    [
        ("impute", SimpleImputer()),
        ("scale", QuantileTransformer(n_quantiles=100, subsample=200_000, random_state=0)),
        (
            "linear_model",
            SGDClassifier(
                loss="log",
                average=True,
                class_weight="balanced",
                n_jobs=5,
                random_state=0,
            ),
        ),
    ]
)

# 1. Univariate feature selection (filter methods)

In [8]:
from sklearn.feature_selection import SelectFdr, SelectFpr, SelectFwe, SelectPercentile, f_classif

selector_obj = [
    SelectFdr(f_classif),
    SelectFpr(f_classif),
    SelectFwe(f_classif),
]

X = df_feats[feat_cols].values
X = SimpleImputer().fit_transform(X)

for selector in selector_obj:
    selector.fit(X, df_feats["label"])
    print("Selector: ", selector, " % selected features: ", round(np.sum(selector.get_support()) / X.shape[1], 4))

Selector:  SelectFdr()  % selected features:  0.9857
Selector:  SelectFpr()  % selected features:  0.9857
Selector:  SelectFwe()  % selected features:  0.9857


All these techniques throw away 1.5% of the features, which will have minimal impact on model performance.

In [9]:
selector = selector_obj[0]
X_selected = selector.transform(X)

# Total of 10 folds
gkfold = StratifiedGroupKFold(n_splits=10)

print("----" * 10)
print("NO Feature selection")
print(X.shape)
cv = gkfold.split(X, df_feats["label"], groups=df_feats.patient_id)
res = cross_validate(
    pipe,
    X,
    df_feats["label"],
    scoring=["f1_macro", "balanced_accuracy", "accuracy", "neg_log_loss"],
    cv=cv,
    n_jobs=25,
    return_train_score=True,
    return_estimator=True,
)

print("10-FOLD: TRAIN")
print("  MACRO F1:          ", round(np.mean(res["train_f1_macro"]), 4))
print("  Balanced accuracy: ", round(np.mean(res["train_balanced_accuracy"]), 4))
print("  Accuracy:          ", round(np.mean(res["train_accuracy"]), 4))
print("  Log loss:          ", round(np.mean(-1 * res["train_neg_log_loss"]), 4))

print("10-FOLD: TEST")
print("  MACRO F1:          ", round(np.mean(res["test_f1_macro"]), 4))
print("  Balanced accuracy: ", round(np.mean(res["test_balanced_accuracy"]), 4))
print("  Accuracy:          ", round(np.mean(res["test_accuracy"]), 4))
print("  Log loss:          ", round(np.mean(-1 * res["test_neg_log_loss"]), 4))

print("----" * 10)
print("Feature selection")
print(X_selected.shape)
cv = gkfold.split(X_selected, df_feats["label"], groups=df_feats.patient_id)
res = cross_validate(
    pipe,
    X_selected,
    df_feats["label"],
    scoring=["f1_macro", "balanced_accuracy", "accuracy", "neg_log_loss"],
    cv=cv,
    n_jobs=25,
    return_train_score=True,
    return_estimator=True,
)

print("10-FOLD: TRAIN")
print("  MACRO F1:          ", round(np.mean(res["train_f1_macro"]), 4))
print("  Balanced accuracy: ", round(np.mean(res["train_balanced_accuracy"]), 4))
print("  Accuracy:          ", round(np.mean(res["train_accuracy"]), 4))
print("  Log loss:          ", round(np.mean(-1 * res["train_neg_log_loss"]), 4))

print("10-FOLD: TEST")
print("  MACRO F1:          ", round(np.mean(res["test_f1_macro"]), 4))
print("  Balanced accuracy: ", round(np.mean(res["test_balanced_accuracy"]), 4))
print("  Accuracy:          ", round(np.mean(res["test_accuracy"]), 4))
print("  Log loss:          ", round(np.mean(-1 * res["test_neg_log_loss"]), 4))

----------------------------------------
NO Feature selection
(195168, 1048)
10-FOLD: TRAIN
  MACRO F1:           0.7948
  Balanced accuracy:  0.8105
  Accuracy:           0.8407
  Log loss:           0.5406
10-FOLD: TEST
  MACRO F1:           0.7698
  Balanced accuracy:  0.7853
  Accuracy:           0.8209
  Log loss:           0.6601
----------------------------------------
Feature selection
(195168, 1033)
10-FOLD: TRAIN
  MACRO F1:           0.7948
  Balanced accuracy:  0.8103
  Accuracy:           0.8407
  Log loss:           0.5407
10-FOLD: TEST
  MACRO F1:           0.7693
  Balanced accuracy:  0.7847
  Accuracy:           0.8205
  Log loss:           0.6606


# 2. Wrapper based feature selection

In [10]:
import sys
sys.path.append("../../../powershap/")

In [11]:
from powershap import PowerShap

X = df_feats[feat_cols].values
X = SimpleImputer().fit_transform(X)

selector = PowerShap()
selector.fit(X, df_feats["label"], n_jobs=25)

100%|██████████| 10/10 [26:18<00:00, 157.81s/it]


PowerShap(model=<catboost.core.CatBoostClassifier object at 0x7fe189179f10>)

In [12]:
X_selected = selector.transform(X)
print("% features selected", round(X_selected.shape[1] / X.shape[1], 4))
print(X_selected.shape, X.shape)

% features selected 0.1403
(195168, 147) (195168, 1048)


In [13]:
# Total of 10 folds
gkfold = StratifiedGroupKFold(n_splits=10)

print("----" * 10)
print("NO Feature selection")
print(X.shape)
cv = gkfold.split(X, df_feats["label"], groups=df_feats.patient_id)
res = cross_validate(
    pipe,
    X,
    df_feats["label"],
    scoring=["f1_macro", "balanced_accuracy", "accuracy", "neg_log_loss"],
    cv=cv,
    n_jobs=25,
    return_train_score=True,
    return_estimator=True,
)

print("10-FOLD: TRAIN")
print("  MACRO F1:          ", round(np.mean(res["train_f1_macro"]), 4))
print("  Balanced accuracy: ", round(np.mean(res["train_balanced_accuracy"]), 4))
print("  Accuracy:          ", round(np.mean(res["train_accuracy"]), 4))
print("  Log loss:          ", round(np.mean(-1 * res["train_neg_log_loss"]), 4))

print("10-FOLD: TEST")
print("  MACRO F1:          ", round(np.mean(res["test_f1_macro"]), 4))
print("  Balanced accuracy: ", round(np.mean(res["test_balanced_accuracy"]), 4))
print("  Accuracy:          ", round(np.mean(res["test_accuracy"]), 4))
print("  Log loss:          ", round(np.mean(-1 * res["test_neg_log_loss"]), 4))

print("----" * 10)
print("Feature selection")
print(X_selected.shape)
cv = gkfold.split(X_selected, df_feats["label"], groups=df_feats.patient_id)
res = cross_validate(
    pipe,
    X_selected,
    df_feats["label"],
    scoring=["f1_macro", "balanced_accuracy", "accuracy", "neg_log_loss"],
    cv=cv,
    n_jobs=25,
    return_train_score=True,
    return_estimator=True,
)

print("10-FOLD: TRAIN")
print("  MACRO F1:          ", round(np.mean(res["train_f1_macro"]), 4))
print("  Balanced accuracy: ", round(np.mean(res["train_balanced_accuracy"]), 4))
print("  Accuracy:          ", round(np.mean(res["train_accuracy"]), 4))
print("  Log loss:          ", round(np.mean(-1 * res["train_neg_log_loss"]), 4))

print("10-FOLD: TEST")
print("  MACRO F1:          ", round(np.mean(res["test_f1_macro"]), 4))
print("  Balanced accuracy: ", round(np.mean(res["test_balanced_accuracy"]), 4))
print("  Accuracy:          ", round(np.mean(res["test_accuracy"]), 4))
print("  Log loss:          ", round(np.mean(-1 * res["test_neg_log_loss"]), 4))

----------------------------------------
NO Feature selection
(195168, 1048)
10-FOLD: TRAIN
  MACRO F1:           0.7948
  Balanced accuracy:  0.8105
  Accuracy:           0.8407
  Log loss:           0.5406
10-FOLD: TEST
  MACRO F1:           0.7698
  Balanced accuracy:  0.7853
  Accuracy:           0.8209
  Log loss:           0.6601
----------------------------------------
Feature selection
(195168, 147)
10-FOLD: TRAIN
  MACRO F1:           0.7767
  Balanced accuracy:  0.7927
  Accuracy:           0.8268
  Log loss:           0.4645
10-FOLD: TEST
  MACRO F1:           0.759
  Balanced accuracy:  0.7754
  Accuracy:           0.8128
  Log loss:           0.5103


Here we observe a net negative effect when applying feature selection